# <font color=blue>NCAA Basketball Championship 2019 Results Prediction</font>

## <font color=purple>Problem</font>

This project is a response to the [NCAA Basketball Championship 2019 Kaggle Challenge](https://www.kaggle.com/c/mens-machine-learning-competition-2019) which relies on results of past seasons' tournaments to predict the results for 2019 season using Machine Learning.

## <font color=purple>Problem Solving Approach</font>

The approach used involves the following steps:
1. **Data Loading & Entity Identification**
loading all the [datasets available](https://www.kaggle.com/c/mens-machine-learning-competition-2019/data) and identifying all the data entities involved in thethe relationships between entities 
2. **Data Cleaning & Wrangling**
performing cleanups and appropriate data conversions,  merging two or more entities into one, identifying appropriate temporal attributes for each entity (date-time indexes)
3. **[Automatic Feature Engineering](https://medium.com/@rrfd/simple-automatic-feature-engineering-using-featuretools-in-python-for-classification-b1308040e183)** using [FeatureTools](https://docs.featuretools.com/index.html)
4. **Feature Selection**
5. **Modeling**
using multiple techniques, hyperparameter tuning and model evaluation

## <font color=purple>Imports & Initialization</font>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import time


from joblib import dump, load
import pickle
import os
from tqdm import tqdm

import sys
sys.path.append('..')
from src.utils.plotting import plot_roc, bar_plot_maker

import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV

import featuretools as ft

pd.options.mode.use_inf_as_na = True

data_folder = '/media/siri/78C6823EC681FD1E/minio/data/kaggle/ncaa/'

## <font color=purple>Entity Relationship Diagram</font>

<img src="../img/ncaa_ERD.png" width=905 height=996 align='left'></img>

## <font color=purple>Data Loading & Entity Identification</font>

### <font color=green>1. Teams</font>

In [6]:
teams_df = pd.read_csv(data_folder+'Teams.csv', dtype={'TeamID':np.int64, 'FirstD1Season':np.int64, 'LastD1Season':np.int64})
teams_df.head()

,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2019
1,1102,Air Force,1985,2019
2,1103,Akron,1985,2019
3,1104,Alabama,1985,2019
4,1105,Alabama A&M,2000,2019


In [7]:
teams_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 4 columns):
TeamID           366 non-null int64
TeamName         366 non-null object
FirstD1Season    366 non-null int64
LastD1Season     366 non-null int64
dtypes: int64(3), object(1)
memory usage: 11.5+ KB


### <font color=green>2. Tournament Seeds</font>

In [8]:
tourney_seeds_df = pd.read_csv(data_folder+'NCAATourneySeeds.csv', dtype={'TeamID':np.int64, 
                                                                          'Season':np.int64})
tourney_seeds_df.head()

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374


In [9]:
tourney_seeds_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2218 entries, 0 to 2217
Data columns (total 3 columns):
Season    2218 non-null int64
Seed      2218 non-null object
TeamID    2218 non-null int64
dtypes: int64(2), object(1)
memory usage: 52.1+ KB


### <font color=green>3. Seasons</font>

In [10]:
seasons_df = pd.read_csv(data_folder+'Seasons.csv', dtype={'Season':np.int64}, parse_dates=['DayZero'])
seasons_df.head()

,Season,DayZero,RegionW,RegionX,RegionY,RegionZ
0,1985,1984-10-29,East,West,Midwest,Southeast
1,1986,1985-10-28,East,Midwest,Southeast,West
2,1987,1986-10-27,East,Southeast,Midwest,West
3,1988,1987-11-02,East,Midwest,Southeast,West
4,1989,1988-10-31,East,West,Midwest,Southeast


### <font color=green>4. Regular Season Games - Compact Results</font>

In [11]:
reg_comp_df = pd.read_csv(data_folder+'RegularSeasonCompactResults.csv', dtype={'Season':np.int64, 
                                                                                'DayNum':np.int64, 
                                                                                'WTeamID':np.int64, 
                                                                                'LTeamID':np.int64, 
                                                                                'WScore':np.float64, 
                                                                                'LScore':np.float64, 
                                                                                'NumOT':np.int64})
reg_comp_df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,20,1228,81.0,1328,64.0,N,0
1,1985,25,1106,77.0,1354,70.0,H,0
2,1985,25,1112,63.0,1223,56.0,H,0
3,1985,25,1165,70.0,1432,54.0,H,0
4,1985,25,1192,86.0,1447,74.0,H,0


In [12]:
reg_comp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156089 entries, 0 to 156088
Data columns (total 8 columns):
Season     156089 non-null int64
DayNum     156089 non-null int64
WTeamID    156089 non-null int64
WScore     156089 non-null float64
LTeamID    156089 non-null int64
LScore     156089 non-null float64
WLoc       156089 non-null object
NumOT      156089 non-null int64
dtypes: float64(2), int64(5), object(1)
memory usage: 9.5+ MB


### <font color=green>5. Regular Season Games - Detailed Results</font>

In [13]:
reg_det_df = pd.read_csv(data_folder+'RegularSeasonDetailedResults.csv', dtype={'Season':np.int64, 
                                                                                'DayNum':np.int64, 
                                                                                'WTeamID':np.int64, 
                                                                                'LTeamID':np.int64, 
                                                                                'WScore':np.float64, 
                                                                                'LScore':np.float64, 
                                                                                'NumOT':np.int64})
reg_det_df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68.0,1328,62.0,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70.0,1393,63.0,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73.0,1437,61.0,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56.0,1457,50.0,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77.0,1208,71.0,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


### <font color=green>6. NCAA Tournament Games - Compact Results</font>

In [14]:
ncaa_comp_df = pd.read_csv(data_folder+'NCAATourneyCompactResults.csv', dtype={'Season':np.int64, 
                                                                                'DayNum':np.int64, 
                                                                                'WTeamID':np.int64, 
                                                                                'LTeamID':np.int64, 
                                                                                'WScore':np.float64, 
                                                                                'LScore':np.float64, 
                                                                                'NumOT':np.int64})
ncaa_comp_df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,136,1116,63.0,1234,54.0,N,0
1,1985,136,1120,59.0,1345,58.0,N,0
2,1985,136,1207,68.0,1250,43.0,N,0
3,1985,136,1229,58.0,1425,55.0,N,0
4,1985,136,1242,49.0,1325,38.0,N,0


### <font color=green>7. NCAA Tournament Games - Detailed Results</font>

In [15]:
ncaa_det_df = pd.read_csv(data_folder+'NCAATourneyDetailedResults.csv', dtype={'Season':np.int64, 
                                                                                'DayNum':np.int64, 
                                                                                'WTeamID':np.int64, 
                                                                                'LTeamID':np.int64, 
                                                                                'WScore':np.float64, 
                                                                                'LScore':np.float64, 
                                                                                'NumOT':np.int64})
ncaa_det_df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,134,1421,92.0,1411,84.0,N,1,32,69,...,31,14,31,17,28,16,15,5,0,22
1,2003,136,1112,80.0,1436,51.0,N,0,31,66,...,16,7,7,8,26,12,17,10,3,15
2,2003,136,1113,84.0,1272,71.0,N,0,31,59,...,28,14,21,20,22,11,12,2,5,18
3,2003,136,1141,79.0,1166,73.0,N,0,29,53,...,17,12,17,14,17,20,21,6,6,21
4,2003,136,1143,76.0,1301,74.0,N,1,27,64,...,21,15,20,10,26,16,14,5,8,19


### <font color=green>8. Secondary Tournament Games Results</font>

In [24]:
sec_comp_df = pd.read_csv(data_folder+'SecondaryTourneyCompactResults.csv', dtype={'Season':np.int64, 
                                                                                'DayNum':np.int64, 
                                                                                'WTeamID':np.int64, 
                                                                                'LTeamID':np.int64, 
                                                                                'WScore':np.float64, 
                                                                                'LScore':np.float64, 
                                                                                'NumOT':np.int64})
sec_comp_df.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,SecondaryTourney
0,1985,136,1151,67.0,1155,65.0,H,0,NIT
1,1985,136,1153,77.0,1245,61.0,H,0,NIT
2,1985,136,1201,79.0,1365,76.0,H,0,NIT
3,1985,136,1231,79.0,1139,57.0,H,0,NIT
4,1985,136,1249,78.0,1222,71.0,H,0,NIT


In [34]:
sec_comp_df.SecondaryTourney.unique()

array(['NIT', 'CBI', 'CIT', 'V16', 'NIT '], dtype=object)

In [35]:
sec_comp_df['SecondaryTourney'] = sec_comp_df['SecondaryTourney'].str.strip()
sec_comp_df['SecondaryTourney'].unique()

array(['NIT', 'CBI', 'CIT', 'V16'], dtype=object)

In [25]:
sec_comp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1551 entries, 0 to 1550
Data columns (total 9 columns):
Season              1551 non-null int64
DayNum              1551 non-null int64
WTeamID             1551 non-null int64
WScore              1551 non-null float64
LTeamID             1551 non-null int64
LScore              1551 non-null float64
WLoc                1551 non-null object
NumOT               1551 non-null int64
SecondaryTourney    1551 non-null object
dtypes: float64(2), int64(5), object(2)
memory usage: 109.1+ KB


### <font color=green>9. Geography / Game Locations</font>

In [16]:
game_cities_df = pd.read_csv(data_folder+'GameCities.csv', dtype={'Season':np.int64, 
                                                                    'DayNum':np.int64, 
                                                                    'WTeamID':np.int64, 
                                                                    'LTeamID':np.int64, 
                                                                    'CityID':np.int64})
game_cities_df.head()

,Season,DayNum,WTeamID,LTeamID,CRType,CityID
0,2010,7,1143,1293,Regular,4027
1,2010,7,1314,1198,Regular,4061
2,2010,7,1326,1108,Regular,4080
3,2010,7,1393,1107,Regular,4340
4,2010,9,1143,1178,Regular,4027


In [17]:
game_cities_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49235 entries, 0 to 49234
Data columns (total 6 columns):
Season     49235 non-null int64
DayNum     49235 non-null int64
WTeamID    49235 non-null int64
LTeamID    49235 non-null int64
CRType     49235 non-null object
CityID     49235 non-null int64
dtypes: int64(5), object(1)
memory usage: 2.3+ MB


### <font color=green>10. Cities</font>

In [18]:
cities_df = pd.read_csv(data_folder+'Cities.csv', dtype={'CityID':np.int64})
cities_df.head()

,CityID,City,State
0,4001,Abilene,TX
1,4002,Akron,OH
2,4003,Albany,NY
3,4004,Albuquerque,NM
4,4005,Allentown,PA


In [19]:
cities_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421 entries, 0 to 420
Data columns (total 3 columns):
CityID    421 non-null int64
City      421 non-null object
State     421 non-null object
dtypes: int64(1), object(2)
memory usage: 9.9+ KB


### <font color=green>11. Massey Rankings</font>

In [20]:
mass_rnks_df = pd.read_csv(data_folder+'MasseyOrdinals.csv', encoding = "ISO-8859-1") #, dtype={'Season':np.int64,'RankingDayNum':np.int64,'TeamID':np.int64,'OrdinalRank':np.int64})
mass_rnks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3492320 entries, 0 to 3492319
Data columns (total 5 columns):
Season           int64
RankingDayNum    int64
SystemName       object
TeamID           int64
OrdinalRank      int64
dtypes: int64(4), object(1)
memory usage: 133.2+ MB


In [22]:
mass_rnks_df['Season'] = pd.to_numeric(mass_rnks_df['Season'], errors='coerce', downcast='integer')
mass_rnks_df['RankingDayNum'] = pd.to_numeric(mass_rnks_df['RankingDayNum'], errors='coerce', downcast='integer')
mass_rnks_df['TeamID'] = pd.to_numeric(mass_rnks_df['TeamID'], errors='coerce', downcast='integer')
mass_rnks_df['OrdinalRank'] = pd.to_numeric(mass_rnks_df['OrdinalRank'], errors='coerce', downcast='integer')
#mass_rnks_df.loc[mass_rnks_df.isna().any(axis=1)]

mass_rnks_df = mass_rnks_df.dropna(subset=['Season', 'RankingDayNum', 'TeamID', 'OrdinalRank'])
mass_rnks_df['Season'] = mass_rnks_df['Season'].astype(np.int64)
mass_rnks_df['RankingDayNum'] = mass_rnks_df['RankingDayNum'].astype(np.int64)
mass_rnks_df['TeamID'] = mass_rnks_df['TeamID'].astype(np.int64)
mass_rnks_df['OrdinalRank'] = mass_rnks_df['OrdinalRank'].astype(np.int64)
mass_rnks_df.info()

mass_rnks_df = mass_rnks_df.dropna(subset=['Season', 'RankingDayNum', 'TeamID', 'OrdinalRank'])
mass_rnks_df['Season'] = mass_rnks_df['Season'].astype(np.int64)
mass_rnks_df['RankingDayNum'] = mass_rnks_df['RankingDayNum'].astype(np.int64)
mass_rnks_df['TeamID'] = mass_rnks_df['TeamID'].astype(np.int64)
mass_rnks_df['OrdinalRank'] = mass_rnks_df['OrdinalRank'].astype(np.int64)
mass_rnks_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3492320 entries, 0 to 3492319
Data columns (total 5 columns):
Season           int64
RankingDayNum    int64
SystemName       object
TeamID           int64
OrdinalRank      int64
dtypes: int64(4), object(1)
memory usage: 159.9+ MB


### <font color=green>12. Game Events (Play-by-play event details for each game)</font>

In [26]:
#### 2010
ev_10_df = pd.read_csv(data_folder+'Events_2010.csv')
pl_10_df = pd.read_csv(data_folder+'Players_2010.csv')

#### 2011
ev_11_df = pd.read_csv(data_folder+'Events_2011.csv')
pl_11_df = pd.read_csv(data_folder+'Players_2011.csv')

#### 2012
ev_12_df = pd.read_csv(data_folder+'Events_2012.csv')
pl_12_df = pd.read_csv(data_folder+'Players_2012.csv')

#### 2013
ev_13_df = pd.read_csv(data_folder+'Events_2013.csv')
pl_13_df = pd.read_csv(data_folder+'Players_2013.csv')

#### 2014
ev_14_df = pd.read_csv(data_folder+'Events_2014.csv')
pl_14_df = pd.read_csv(data_folder+'Players_2014.csv')

#### 2015
ev_15_df = pd.read_csv(data_folder+'Events_2015.csv')
pl_15_df = pd.read_csv(data_folder+'Players_2015.csv')

#### 2016
ev_16_df = pd.read_csv(data_folder+'Events_2016.csv')
pl_16_df = pd.read_csv(data_folder+'Players_2016.csv')

#### 2017
ev_17_df = pd.read_csv(data_folder+'Events_2017.csv')
pl_17_df = pd.read_csv(data_folder+'Players_2017.csv')

#### 2018
ev_18_df = pd.read_csv(data_folder+'Events_2018.csv')
pl_18_df = pd.read_csv(data_folder+'Players_2018.csv')

ev_18_df.head()

,EventID,Season,DayNum,WTeamID,LTeamID,WPoints,LPoints,ElapsedSeconds,EventTeamID,EventPlayerID,EventType
0,27543273,2018,11,1104,1272,0,0,15,1104,648148,assist
1,27543274,2018,11,1104,1272,3,0,15,1104,648149,made3_jump
2,27543275,2018,11,1104,1272,0,0,39,1272,650613,turnover
3,27543276,2018,11,1104,1272,0,0,43,1104,648156,miss2_jump
4,27543277,2018,11,1104,1272,0,0,43,1104,648148,reb_off


##### <font color=darkblue>Merge all Events into one dataframe</font>

In [27]:
events_df = pd.concat([ev_18_df, ev_17_df, ev_16_df, ev_15_df, ev_14_df, ev_13_df, ev_12_df, ev_11_df, ev_10_df], ignore_index=True)
events_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22991175 entries, 0 to 22991174
Data columns (total 11 columns):
EventID           int64
Season            int64
DayNum            int64
WTeamID           int64
LTeamID           int64
WPoints           int64
LPoints           object
ElapsedSeconds    int64
EventTeamID       int64
EventPlayerID     object
EventType         object
dtypes: int64(8), object(3)
memory usage: 1.9+ GB


##### <font color=darkblue>Convert data to appropriate types</font>

In [28]:
events_df = events_df.drop(columns=['EventID'])
events_df.index = events_df.index.rename('EventID')
events_df['EventPlayerID'] = pd.to_numeric(events_df['EventPlayerID'], downcast='integer',errors='coerce')
events_df['LPoints'] = pd.to_numeric(events_df['EventPlayerID'], errors='coerce')
events_df['Season'] = pd.to_numeric(events_df['Season'], errors='coerce')
events_df = events_df.dropna(subset=['EventPlayerID','LPoints','Season'])

events_df['Season'] = events_df['Season'].astype('int64')
events_df['DayNum'] = events_df['DayNum'].astype('int64')
events_df['WTeamID'] = events_df['WTeamID'].astype('int64')
events_df['LTeamID'] = events_df['LTeamID'].astype('int64')
events_df['WPoints'] = events_df['WPoints'].astype('float64')
events_df['LPoints'] = events_df['WPoints'].astype('float64')
events_df['ElapsedSeconds'] = events_df['ElapsedSeconds'].astype('int64')
events_df['EventTeamID'] = events_df['EventTeamID'].astype('int64')
events_df['EventPlayerID'] = events_df['EventPlayerID'].astype('int64')
events_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22991174 entries, 0 to 22991174
Data columns (total 10 columns):
Season            int64
DayNum            int64
WTeamID           int64
LTeamID           int64
WPoints           float64
LPoints           float64
ElapsedSeconds    int64
EventTeamID       int64
EventPlayerID     int64
EventType         object
dtypes: float64(2), int64(7), object(1)
memory usage: 1.9+ GB


### <font color=green>13. Players</font>

In [30]:
players_df = pd.concat([pl_18_df, pl_17_df, pl_16_df, pl_15_df, pl_14_df, pl_13_df, pl_12_df, pl_11_df, pl_10_df], ignore_index=True)
players_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48179 entries, 0 to 48178
Data columns (total 4 columns):
PlayerID      48179 non-null int64
Season        48179 non-null int64
TeamID        48179 non-null int64
PlayerName    48179 non-null object
dtypes: int64(3), object(1)
memory usage: 1.5+ MB


## <font color=purple>Wrangle Data and Refactor Entities</font>
(as per new Entity Relationship Diagram shown below)

<img src="../img/ncaa_ERD_refactored.png" width=‪574 height=807 align='left'></img>

### <font color=darkgreen>Merge Tournament, Regular and Secondary Games (compact results) to create a single games dataframe</font>
##### <font color=darkblue>Add a TourneyName column to identify games as Tournament (NCAA), Reguler (REG) or Secondary(others)</font>

In [36]:
new_col = 'TourneyName'
ncaa_comp_df[new_col] = 'NCAA'
reg_comp_df[new_col] = 'REG'
sec_comp_df.rename(index=str, columns={'SecondaryTourney': new_col}, inplace=True)

games_comp_df = pd.concat([ncaa_comp_df, reg_comp_df, sec_comp_df], ignore_index=True)

games_comp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159824 entries, 0 to 159823
Data columns (total 9 columns):
Season         159824 non-null int64
DayNum         159824 non-null int64
WTeamID        159824 non-null int64
WScore         159824 non-null float64
LTeamID        159824 non-null int64
LScore         159824 non-null float64
WLoc           159824 non-null object
NumOT          159824 non-null int64
TourneyName    159824 non-null object
dtypes: float64(2), int64(5), object(2)
memory usage: 11.0+ MB


### <font color=darkgreen>Merge Tourney and Regular Games (detailed results)</font>
##### <font color=darkblue>Add foreign key to index of games (merged compact results)</font>

In [46]:
games_det_df = pd.concat([ncaa_det_df, reg_det_df], ignore_index=True)

games_det_df = games_det_df.set_index(['Season', 'DayNum', 'WTeamID', 'LTeamID']).sort_index()

games_comp_df = games_comp_df.reset_index().rename(index=str, columns={'index': 'GameID'})

games_comp_df = games_comp_df.set_index(['Season', 'DayNum', 'WTeamID', 'LTeamID']).sort_index()

games_det_df = games_det_df.merge(games_comp_df, left_index=True, right_index=True, how='left')

games_det_df = games_det_df.drop(columns=['WScore_y', 'LScore_y', 'WLoc_y', 'NumOT_y']).\
                   rename(index=str, columns={'WScore_x':'WScore', 'LScore_x':'LScore', 'WLoc_x':'WLoc', 'NumOT_x':'NumOT'}) 

games_det_df = games_det_df.reset_index().set_index('GameID')

games_det_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83089 entries, 76232 to 2183
Data columns (total 35 columns):
Season         83089 non-null object
DayNum         83089 non-null object
WTeamID        83089 non-null object
LTeamID        83089 non-null object
WScore         83089 non-null float64
LScore         83089 non-null float64
WLoc           83089 non-null object
NumOT          83089 non-null int64
WFGM           83089 non-null int64
WFGA           83089 non-null int64
WFGM3          83089 non-null int64
WFGA3          83089 non-null int64
WFTM           83089 non-null int64
WFTA           83089 non-null int64
WOR            83089 non-null int64
WDR            83089 non-null int64
WAst           83089 non-null int64
WTO            83089 non-null int64
WStl           83089 non-null int64
WBlk           83089 non-null int64
WPF            83089 non-null int64
LFGM           83089 non-null int64
LFGA           83089 non-null int64
LFGM3          83089 non-null int64
LFGA3          

##### <font color=darkblue>Drop redundant columns in detail games</font>

In [48]:
games_det_df = games_det_df.drop(columns=['Season','DayNum','WTeamID', 'LTeamID'])
games_det_df.head()

,WScore,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,...,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,TourneyName
GameID,,,,,,,,,,,,,,,,,,,,,
76232,68.0,62.0,N,0,27,58,3,14,11,18,...,16,22,10,22,8,18,9,2,20,REG
76233,70.0,63.0,N,0,26,62,8,20,10,19,...,9,20,20,25,7,12,8,6,16,REG
76234,73.0,61.0,N,0,24,58,8,18,17,29,...,14,23,31,22,9,12,2,5,23,REG
76235,56.0,50.0,N,0,18,38,3,9,17,31,...,8,15,17,20,9,19,4,3,23,REG
76236,77.0,71.0,N,0,30,61,6,14,11,13,...,17,27,21,15,12,10,7,1,14,REG


In [47]:
games_comp_df = games_comp_df.reset_index().set_index('GameID')
games_comp_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159824 entries, 2184 to 2183
Data columns (total 9 columns):
Season         159824 non-null int64
DayNum         159824 non-null int64
WTeamID        159824 non-null int64
LTeamID        159824 non-null int64
WScore         159824 non-null float64
LScore         159824 non-null float64
WLoc           159824 non-null object
NumOT          159824 non-null int64
TourneyName    159824 non-null object
dtypes: float64(2), int64(5), object(2)
memory usage: 12.2+ MB


### <font color=darkgreen>Add GameID as foreign key to Events</font>
##### <font color=darkblue>Remove redundant columns (Season, DayNum, WTeamID and LTeamID)</font>

In [55]:
events_df = events_df.reset_index().set_index(['Season', 'DayNum', 'WTeamID', 'LTeamID']).sort_index()

events_df = events_df.merge(games_comp_df.reset_index().set_index(['Season', 'DayNum', 'WTeamID', 'LTeamID']).sort_index(), 
                            left_index=True, right_index=True, how='left')

events_df = events_df.loc[events_df.GameID.notna()].reset_index()
events_df['GameID'] = events_df['GameID'].astype(np.int64)
events_df = events_df.set_index('GameID')

events_df = events_df.drop(columns=['Season', 'DayNum', 'TourneyName','WScore', 'LScore', 'WLoc', 'NumOT'])
events_df.head()

,WTeamID,LTeamID,EventID,WPoints,LPoints,ElapsedSeconds,EventTeamID,EventPlayerID,EventType
GameID,,,,,,,,,
110306,1143,1293,20488545,0.0,0.0,0,1143,600578,sub_in
110306,1143,1293,20488546,0.0,0.0,0,1143,600584,sub_in
110306,1143,1293,20488547,0.0,0.0,0,1143,600585,sub_in
110306,1143,1293,20488548,0.0,0.0,10,1143,600581,miss2_lay
110306,1143,1293,20488549,0.0,0.0,10,1143,600581,reb_off


### <font color=darkgreen>Add GameID as foreign key to Geography</font>
##### <font color=darkblue>Remove redundant columns (Season, DayNum, WTeamID and LTeamID)</font>

In [59]:
game_cities_df = game_cities_df.set_index(['Season', 'DayNum', 'WTeamID', 'LTeamID']).sort_index()
game_cities_df = game_cities_df.merge(games_comp_df.reset_index().set_index(['Season', 'DayNum', 'WTeamID', 'LTeamID']).sort_index(), 
                            left_index=True, right_index=True, how='left')

game_cities_df = game_cities_df.loc[game_cities_df.GameID.notna()].reset_index().set_index('GameID')

game_cities_df = game_cities_df.drop(columns=['Season', 'DayNum', 'WTeamID', 'LTeamID', 'TourneyName'])
game_cities_df.head()

,CRType,CityID,WScore,LScore,WLoc,NumOT
GameID,,,,,,
110306,Regular,4027,75.0,70.0,H,0
110307,Regular,4061,88.0,72.0,H,0
110308,Regular,4080,100.0,60.0,H,0
110309,Regular,4340,75.0,43.0,H,0
110310,Regular,4027,95.0,61.0,H,0


### <font color=darkgreen>Add region information to tourney_seeds</font>
##### <font color=darkblue>Join with Seasons table</font>

In [61]:
def change_region(row):
    region_col = 'Region'+row.Region
    row.Region = row[region_col]
    return row

import re
def make_seed_numeric(row):
    m = re.search('\d+', str(row['Seed']))
    row.Seed = m.group(0)
    return row

regionwise_seeds = tourney_seeds_df
regionwise_seeds['Region'] = regionwise_seeds['Seed'].str[0]
#regionwise_seeds.Region.unique()

regionwise_seeds = regionwise_seeds.merge(seasons_df, on='Season', how='left')

regionwise_seeds = regionwise_seeds.apply(change_region, axis=1)

regionwise_seeds = regionwise_seeds.drop(columns=['RegionW','RegionX','RegionY', 'RegionZ']).\
                    rename(columns={'DayZero':'Date'})

regionwise_seeds = regionwise_seeds.apply(make_seed_numeric, axis=1).reset_index().\
                    rename(index=str, columns={'index':'SeedID'}).set_index('SeedID')
regionwise_seeds.Seed = pd.to_numeric(regionwise_seeds.Seed)
regionwise_seeds.head()

,Season,Seed,TeamID,Region,Date
SeedID,,,,,
0,1985,1,1207,East,1984-10-29
1,1985,2,1210,East,1984-10-29
2,1985,3,1228,East,1984-10-29
3,1985,4,1260,East,1984-10-29
4,1985,5,1374,East,1984-10-29


### <font color=darkgreen>Add appropriate Indexes to Entities</font>

In [66]:
mass_rnks_df = mass_rnks_df.reset_index().rename(index=str, columns={'index':'MasseyRankID'}).set_index('MasseyRankID')
mass_rnks_df.head()

,Season,RankingDayNum,SystemName,TeamID,OrdinalRank
MasseyRankID,,,,,
0,2003,35,SEL,1102,159
1,2003,35,SEL,1103,229
2,2003,35,SEL,1104,12
3,2003,35,SEL,1105,314
4,2003,35,SEL,1106,260


In [68]:
seasons_df = seasons_df.set_index('Season')
seasons_df.head()

,DayZero,RegionW,RegionX,RegionY,RegionZ
Season,,,,,
1985,1984-10-29,East,West,Midwest,Southeast
1986,1985-10-28,East,Midwest,Southeast,West
1987,1986-10-27,East,Southeast,Midwest,West
1988,1987-11-02,East,Midwest,Southeast,West
1989,1988-10-31,East,West,Midwest,Southeast


In [70]:
teams_df = teams_df.set_index('TeamID')
teams_df.head()

,TeamName,FirstD1Season,LastD1Season
TeamID,,,
1101,Abilene Chr,2014,2019
1102,Air Force,1985,2019
1103,Akron,1985,2019
1104,Alabama,1985,2019
1105,Alabama A&M,2000,2019


In [71]:
cities_df = cities_df.set_index('CityID')
cities_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 421 entries, 4001 to 4443
Data columns (total 2 columns):
City     421 non-null object
State    421 non-null object
dtypes: object(2)
memory usage: 9.9+ KB


In [72]:
players_df = players_df.set_index('PlayerID')
players_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48179 entries, 648095 to 605287
Data columns (total 3 columns):
Season        48179 non-null int64
TeamID        48179 non-null int64
PlayerName    48179 non-null object
dtypes: int64(2), object(1)
memory usage: 1.5+ MB


### <font color=darkgreen>Create Time Indexes</font>

##### <font color=darkblue>Convert Daynum to absolute date in games massey_rankings so that these dates can be used as time-indexes to generate temporal features</font>

In [76]:
games_comp_df = games_comp_df.merge(seasons_df, left_on='Season', right_index=True, how='left').\
        drop(columns=['RegionW','RegionX','RegionY','RegionZ'])
games_comp_df['Date'] = (pd.to_datetime(games_comp_df['DayZero']) + pd.to_timedelta(games_comp_df['DayNum'])).dt.round('D')
games_comp_df = games_comp_df.drop(columns=['DayNum', 'DayZero'])
games_comp_df.head()

,Season,WTeamID,LTeamID,WScore,LScore,WLoc,NumOT,TourneyName,Date
GameID,,,,,,,,,
2184,1985,1228,1328,81.0,64.0,N,0,REG,1984-10-29
2185,1985,1106,1354,77.0,70.0,H,0,REG,1984-10-29
2186,1985,1112,1223,63.0,56.0,H,0,REG,1984-10-29
2187,1985,1165,1432,70.0,54.0,H,0,REG,1984-10-29
2188,1985,1192,1447,86.0,74.0,H,0,REG,1984-10-29


In [78]:
mass_rnks_df = mass_rnks_df.merge(seasons_df, left_on='Season', right_index=True, how='left').\
        drop(columns=['RegionW','RegionX','RegionY','RegionZ'])
mass_rnks_df['Date'] = (pd.to_datetime(mass_rnks_df['DayZero']) + pd.to_timedelta(mass_rnks_df['RankingDayNum'])).dt.round('D')
mass_rnks_df = mass_rnks_df.drop(columns=['RankingDayNum', 'DayZero'])
mass_rnks_df.head()

,Season,SystemName,TeamID,OrdinalRank,Date
MasseyRankID,,,,,
0,2003,SEL,1102,159,2002-11-04
1,2003,SEL,1103,229,2002-11-04
2,2003,SEL,1104,12,2002-11-04
3,2003,SEL,1105,314,2002-11-04
4,2003,SEL,1106,260,2002-11-04


##### <font color=darkblue>Convert ElapsedSeconds to absolute datetime in the entity events so that the new datetime column can be used as time-index to generate temporal features</font>

##### <font color=darkblue>assume a start-time of noon for each game (as start time information is missing)</font>

In [81]:
games_comp_df['Date'] = games_comp_df['Date'] + pd.Timedelta(hours=12) 
games_comp_df.sample(5)

,Season,WTeamID,LTeamID,WScore,LScore,WLoc,NumOT,TourneyName,Date
GameID,,,,,,,,,
111886,2010,1396,1371,71.0,65.0,A,0,REG,2009-11-02 12:00:00
30222,1992,1158,1154,63.0,60.0,A,0,REG,1991-11-04 12:00:00
128272,2013,1252,1190,60.0,56.0,H,0,REG,2012-11-05 12:00:00
37655,1993,1164,1175,80.0,53.0,N,0,REG,1992-11-02 12:00:00
93452,2006,1214,1224,75.0,63.0,A,0,REG,2005-10-31 12:00:00


In [83]:
events_df = events_df.merge(games_comp_df.\
            drop(columns=['Season', 'WTeamID', 'LTeamID', 'TourneyName', 'WScore', 'LScore', 'WLoc', 'NumOT']),
                            left_index=True, right_index=True, how='left')
#events_df.loc[events_df['Date'].isna()]

events_df['Date'] = events_df['Date'] + pd.to_timedelta(events_df['ElapsedSeconds'], unit='s') 
events_df.sample(10)

,WTeamID,LTeamID,EventID,WPoints,LPoints,ElapsedSeconds,EventTeamID,EventPlayerID,EventType,Date
GameID,,,,,,,,,,
131717,1210,1208,10453183,0.0,0.0,1977,1210,622626,reb_def,2013-11-04 12:32:57
113896,1169,1415,22171705,0.0,0.0,1757,1169,600983,foul_pers,2009-11-02 12:29:17
151689,1436,1420,4535012,0.0,0.0,2390,1436,642371,sub_out,2016-10-31 12:39:50
159356,1305,1137,17946200,0.0,0.0,274,1137,610967,sub_out,2011-10-31 12:04:34
110674,1178,1352,20662869,0.0,0.0,760,1178,601127,sub_out,2009-11-02 12:12:40
120068,1220,1423,20082507,0.0,0.0,650,1220,607034,block,2010-11-01 12:10:50
124028,1257,1437,16972131,0.0,0.0,1076,1437,615359,foul_pers,2011-10-31 12:17:56
1860,1295,1328,12856733,0.0,0.0,817,1295,623931,miss2_lay,2013-11-04 12:13:37
134168,1193,1264,11630563,0.0,0.0,927,1264,623456,timeout,2013-11-04 12:15:27


In [84]:
events_df = events_df.reset_index().set_index('EventID')
events_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22991172 entries, 22932956 to 5098592
Data columns (total 10 columns):
GameID            int64
WTeamID           int64
LTeamID           int64
WPoints           float64
LPoints           float64
ElapsedSeconds    int64
EventTeamID       int64
EventPlayerID     int64
EventType         object
Date              datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(6), object(1)
memory usage: 1.9+ GB


## <font color=purple>Further Refactoring to enable Automatic Feature Generation for Games</font>
(based on the Entity Relationship Diagram shown below)

<img src="../img/ncaa_FT_ERD.png" width=‪824 height=867 align='left'></img>

### <font color=darkgreen>Modify Games and Events</font>
##### <font color=darkblue>so that a parent-child (one-to-many) relationship cna be built from Games (using the composite key GameID , Winning TeamID) to Events (using the composite key GameID , Event TeamID)</font>

##### <font color=darkblue>1. merge game_comp_results,  game_det_results and geography</font>

In [94]:
games_comp_df = games_comp_df.merge(games_det_df.drop(columns=['TourneyName','WScore', 'LScore', 'WLoc', 'NumOT']), 
                                    left_index=True, right_index=True, how='left')

games_comp_df = games_comp_df.merge(game_cities_df.drop(columns=['WScore', 'LScore', 'WLoc', 'NumOT']), 
                                    left_index=True, right_index=True, how='left')
del games_det_df, game_cities_df

games_comp_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159824 entries, 2184 to 2183
Data columns (total 37 columns):
Season         159824 non-null int64
WTeamID        159824 non-null int64
LTeamID        159824 non-null int64
WScore         159824 non-null float64
LScore         159824 non-null float64
WLoc           159824 non-null object
NumOT          159824 non-null int64
TourneyName    159824 non-null object
Date           159824 non-null datetime64[ns]
WFGM           83089 non-null float64
WFGA           83089 non-null float64
WFGM3          83089 non-null float64
WFGA3          83089 non-null float64
WFTM           83089 non-null float64
WFTA           83089 non-null float64
WOR            83089 non-null float64
WDR            83089 non-null float64
WAst           83089 non-null float64
WTO            83089 non-null float64
WStl           83089 non-null float64
WBlk           83089 non-null float64
WPF            83089 non-null float64
LFGM           83089 non-null float64
LFGA    

##### <font color=darkblue>2. Copy games_comp_df to games_comp2_df</font>
##### <font color=darkblue>3. Add composite column GameID+WTeamID to games_comp_df and composite col GameID+LTeamID to games_comp2_df</font> 
##### <font color=darkblue>4. Add column Outcome=1 to games_comp_df and column Outcome=0 to games_comp2_df</font>
##### <font color=darkblue>5. Rename W- columns to L- columns in games_comp2_df</font>
##### <font color=darkblue>6. Concatenate games_comp_df and games_comp2_df</font>

In [98]:
def switch_WLoc(wloc):
    if wloc == 'H':
        wloc = 'A'
    elif wloc == 'A':
        wloc = 'H'
    return wloc

games_comp2_df = games_comp_df.copy()
games_comp_df['Game_WTeamID'] = games_comp_df.index.astype(str) +'_'+games_comp_df['WTeamID'].astype(str)
games_comp_df['Outcome'] = 1
games_comp2_df['Game_WTeamID'] = games_comp2_df.index.astype(str) +'_'+games_comp2_df['LTeamID'].astype(str)
games_comp2_df['Outcome'] = 0
games_comp2_df = games_comp2_df.rename(
    columns={'WTeamID':'LTeamID','WScore':'LScore','WFGM':'LFGM','WFGA':'LFGA','WFGA3':'LFGA3','WFGM3':'LFGM3','WFTM':'LFTM',
             'WFTA':'LFTA','WOR':'LOR','WDR':'LDR','WAst':'LAst','WTO':'LTO','WStl':'LStl','WBlk':'LBlk','WPF':'LPF',
             'LTeamID':'WTeamID','LScore':'WScore','LFGM':'WFGM','LFGA':'WFGA','LFGA3':'WFGA3','LFGM3':'WFGM3','LFTM':'WFTM',
             'LFTA':'WFTA','LOR':'WOR','LDR':'WDR','LAst':'WAst','LTO':'WTO','LStl':'WStl','LBlk':'WBlk','LPF':'WPF'})
games_comp2_df['WLoc'] = games_comp2_df['WLoc'].apply(switch_WLoc)

games_comp_df = pd.concat([games_comp_df, games_comp2_df])

del games_comp2_df

games_comp_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 319648 entries, 2184 to 2183
Data columns (total 39 columns):
CRType          98470 non-null object
CityID          98470 non-null float64
Date            319648 non-null datetime64[ns]
Game_WTeamID    319648 non-null object
LAst            166178 non-null float64
LBlk            166178 non-null float64
LDR             166178 non-null float64
LFGA            166178 non-null float64
LFGA3           166178 non-null float64
LFGM            166178 non-null float64
LFGM3           166178 non-null float64
LFTA            166178 non-null float64
LFTM            166178 non-null float64
LOR             166178 non-null float64
LPF             166178 non-null float64
LScore          319648 non-null float64
LStl            166178 non-null float64
LTO             166178 non-null float64
LTeamID         319648 non-null int64
NumOT           319648 non-null int64
Outcome         319648 non-null int64
Season          319648 non-null int64
TourneyName  

##### <font color=darkblue>7. Add composite column GameID+EventTeamID to events_df</font>

In [99]:
events_df['Game_EventTeamID'] = events_df['GameID'].astype(str)+'_'+events_df['EventTeamID'].astype(str)
events_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22991172 entries, 22932956 to 5098592
Data columns (total 11 columns):
GameID              int64
WTeamID             int64
LTeamID             int64
WPoints             float64
LPoints             float64
ElapsedSeconds      int64
EventTeamID         int64
EventPlayerID       int64
EventType           object
Date                datetime64[ns]
Game_EventTeamID    object
dtypes: datetime64[ns](1), float64(2), int64(6), object(2)
memory usage: 2.1+ GB


## <font color=purple>Partition into all data into chunks to enable Parallel Feature Systhesis using Dask</font>

### <font color=green>Split games (games_comp_df) into 300 partitions</font>
##### <font color=darkblue>and create corresponding partitions for other entities as well and save all partitions to disk</font>

In [6]:
total_chunks = 300

In [134]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

def make_partitions(total_chunks):
    games = list(games_comp_df.reset_index()['GameID'].values)
    chunk_size = len(games) // total_chunks
    # loop over all chunks and create partitons for all entities    
    for chunk in tqdm(range(total_chunks)):
        game_chunks = [games[i*chunk_size:(i+1)*chunk_size] for i in range(total_chunks)]
        games_comp_chunk = games_comp_df.loc[games_comp_df.index.isin(game_chunks[chunk])]
        #games_det_chunk = games_det_df.loc[games_det_df.index.isin(game_chunks[chunk])]
        game_events = list(games_comp_chunk['Game_WTeamID'].unique())
        events_chunk = events_df.loc[events_df['Game_EventTeamID'].isin(game_events)]
        # get list of players
        players = list(events_chunk['EventPlayerID'].unique())
        players_chunk = players_df.loc[players_df.index.isin(players)]
        # get list of seasons
        seasons = list(games_comp_chunk['Season'].unique())
        seasons_chunk = seasons_df.loc[seasons_df.index.isin(seasons)]
        # get list of teams
        teams = set(list(games_comp_chunk['WTeamID'].unique()) + list(games_comp_chunk['LTeamID'].unique()))
        teams_chunk = teams_df.loc[teams_df.index.isin(teams)]
        regionwise_seeds_chunk = regionwise_seeds.loc[regionwise_seeds['TeamID'].isin(teams) &
                                                      regionwise_seeds['Season'].isin(seasons)]
        mass_rnks_chunk = mass_rnks_df.loc[mass_rnks_df['TeamID'].isin(teams) &
                                           mass_rnks_df['Season'].isin(seasons)]
        # get list of locations and cities
        #game_cities_chunk = game_cities_df.loc[game_cities_df.index.isin(game_chunks[chunk])]
        cities = list(games_comp_chunk['CityID'].unique())
        cities_chunk = cities_df.loc[cities_df.index.isin(cities)]
        # save all entity chunks to disk
        createFolder(data_folder+'p{}'.format(chunk))
        games_comp_chunk.reset_index().to_parquet(data_folder+'p{}/games_comp'.format(chunk), compression='SNAPPY',object_encoding='utf8')
        events_chunk.reset_index().to_parquet(data_folder+'p{}/events'.format(chunk), compression='SNAPPY',
                                              object_encoding={'GameID':'utf8','WTeamID':'utf8','LTeamID':'utf8',
                                                               'WPoints':'utf8','LPoints':'utf8','ElapsedSeconds':'utf8',
                                                               'EventTeamID':'utf8','EventPlayerID':'utf8','EventType':'json',
                                                               'Date':'utf8','Game_EventTeamID':'utf8'})
        players_chunk.reset_index().to_parquet(data_folder+'p{}/players'.format(chunk), compression='SNAPPY',object_encoding='utf8')
        teams_chunk.reset_index().to_parquet(data_folder+'p{}/teams'.format(chunk), compression='SNAPPY',object_encoding='utf8')
        regionwise_seeds_chunk.reset_index().to_parquet(data_folder+'p{}/regionwise_seeds'.format(chunk), compression='SNAPPY',object_encoding='utf8')
        mass_rnks_chunk.reset_index().to_parquet(data_folder+'p{}/mass_rnks'.format(chunk), compression='SNAPPY',object_encoding='utf8')
        seasons_chunk.reset_index().to_parquet(data_folder+'p{}/seasons'.format(chunk), compression='SNAPPY',object_encoding='utf8')
        cities_chunk.reset_index().to_parquet(data_folder+'p{}/cities'.format(chunk), compression='SNAPPY',object_encoding='utf8')

make_partitions(total_chunks)


 52%|█████▏    | 155/300 [03:11<02:43,  1.13s/it]


100%|██████████| 300/300 [06:14<00:00,  1.86s/it]

## <font color=purple>Deep Feature Synthesis with FeatureTools</font>

In [11]:
#function to be applied to every row in events (if eventteamid is lteamid, switch WPoints and LPoints)
def switch_points(row):
    if row['EventTeamID'] == row['LTeamID']:
        wpoints = row['WPoints']
        lpoints = row['LPoints']
        row['LPoints'] = wpoints
        row['WPoints'] = lpoints
    return row

import featuretools as ft
# entity set

# function to create featuretools entityset and define relationships between entities in the set 
def create_entityset(path_dict):
    chunk = path_dict['chunk']
    path = path_dict['path']
    es = ft.EntitySet()
    games_comp_chunk = pd.read_parquet(path+'/games_comp')
    
    events_chunk = pd.read_parquet(path+'/events')
    events_chunk = events_chunk.apply(switch_points, axis=1)
    players_chunk = pd.read_parquet(path+'/players')
    seasons_chunk = pd.read_parquet(path+'/seasons')
    teams_chunk = pd.read_parquet(path+'/teams')
    regionwise_seeds_chunk = pd.read_parquet(path+'/regionwise_seeds')
    mass_rnks_chunk = pd.read_parquet(path+'/mass_rnks')
    
    cities_chunk = pd.read_parquet(path+'/cities')

    # add Wteams entity
    es = es.entity_from_dataframe(entity_id='WTeams',
                                 dataframe=teams_chunk,
                                 index='TeamID', 
                                 variable_types={
                                     'TeamName':ft.variable_types.Text,
                                     'FirstD1Season':ft.variable_types.Id,
                                     'LastD1Season':ft.variable_types.Id
                                 })
    

    es = es.entity_from_dataframe(entity_id='LTeams',
                                 dataframe=teams_chunk,
                                 index='TeamID', 
                                 variable_types={
                                     'TeamName':ft.variable_types.Text,
                                     'FirstD1Season':ft.variable_types.Id,
                                     'LastD1Season':ft.variable_types.Id
                                 })
    
    es = es.entity_from_dataframe(entity_id='regionwise_seeds',
                                 dataframe=regionwise_seeds_chunk,
                                 index='SeedID', 
                                 time_index = 'Date',
                                 variable_types={
                                     'Season':ft.variable_types.Id,
                                     'Seed':ft.variable_types.Ordinal,
                                     'TeamID':ft.variable_types.Id,
                                     'Region':ft.variable_types.Text
                                 })
    
    es = es.entity_from_dataframe(entity_id='seasons',
                                 dataframe=seasons_chunk,
                                 index='Season',
                                 time_index='DayZero',
                                 variable_types={
                                     'RegionW':ft.variable_types.Text,
                                     'RegionX':ft.variable_types.Text,
                                     'RegionY':ft.variable_types.Text,
                                     'RegionZ':ft.variable_types.Text
                                     #'DayZero':ft.variable_types.Datetime
                                 })
    
    es = es.entity_from_dataframe(entity_id='games',
                                 dataframe=games_comp_chunk,
                                 index='Game_WTeamID',
                                 time_index='Date', 
                                 variable_types={
                                     'Season':ft.variable_types.Id,
                                     'WTeamID':ft.variable_types.Id,
                                     'LTeamID':ft.variable_types.Id,
                                     'TourneyName':ft.variable_types.Categorical
                                     #'Date':ft.variable_types.Datetime
                                 })

    es = es.entity_from_dataframe(entity_id='cities',
                                 dataframe=cities_chunk,
                                 index='CityID',
                                 variable_types={
                                     'State':ft.variable_types.Categorical,
                                     'City':ft.variable_types.Text
                                 })
    
    es = es.entity_from_dataframe(entity_id='massey_rankings',
                                 dataframe=mass_rnks_chunk,
                                 index='MasseyRankID',
                                 time_index='Date', 
                                 variable_types={
                                    'Season':ft.variable_types.Id,
                                     'SystemName':ft.variable_types.Categorical,
                                     'TeamID':ft.variable_types.Id,
                                     'OrdinalRank':ft.variable_types.Ordinal
                                     #'Date':ft.variable_types.Datetime
                                 })
    
    es = es.entity_from_dataframe(entity_id='WTeam_events',
                                 dataframe=events_chunk,
                                 index='EventID',
                                 time_index='Date', 
                                 variable_types={
                                     'WPoints':ft.variable_types.Numeric,
                                     'LPoints':ft.variable_types.Numeric,
                                     'ElapsedSeconds':ft.variable_types.Numeric,
                                     'EventPlayerID':ft.variable_types.Id,
                                     'Game_EventTeamID':ft.variable_types.Id,
                                     'EventType':ft.variable_types.Categorical
                                     #'Date':ft.variable_types.Datetime
                                 })

    es = es.entity_from_dataframe(entity_id='players',
                                 dataframe=players_chunk,
                                 index='PlayerID',
                                 variable_types={
                                     'Season':ft.variable_types.Id,
                                     'TeamID':ft.variable_types.Id,
                                     'PlayerName':ft.variable_types.Text
                                 })
    

    es.add_relationship(ft.Relationship(es['seasons']['Season'],
                                        es['regionwise_seeds']['Season']))
    es.add_relationship(ft.Relationship(es['WTeams']['TeamID'],
                                       es['regionwise_seeds']['TeamID']))

    es.add_relationship(ft.Relationship(es['seasons']['Season'],
                                        es['massey_rankings']['Season']))
    es.add_relationship(ft.Relationship(es['WTeams']['TeamID'],
                                        es['massey_rankings']['TeamID']))

    es.add_relationship(ft.Relationship(es['LTeams']['TeamID'],
                                       es['regionwise_seeds']['TeamID']))

    es.add_relationship(ft.Relationship(es['LTeams']['TeamID'],
                                        es['massey_rankings']['TeamID']))
    
    es.add_relationship(ft.Relationship(es['seasons']['Season'],
                                        es['games']['Season']))
    es.add_relationship(ft.Relationship(es['WTeams']['TeamID'],
                                        es['games']['WTeamID']))
    es.add_relationship(ft.Relationship(es['LTeams']['TeamID'],
                                        es['games']['LTeamID']))

    es.add_relationship(ft.Relationship(es['games']['Game_WTeamID'],
                                        es['WTeam_events']['Game_EventTeamID']))
    es.add_relationship(ft.Relationship(es['players']['PlayerID'],
                                        es['WTeam_events']['EventPlayerID']))

    # save to disk
    #pickle.dump(es, open(data_folder+'p{}/es.p'.format(chunk), 'wb'))
    return {'es':es, 'chunk':chunk}

def compute_features(es_dict, feature_defs, set_number=1):
    es = es_dict['es']
    chunk = es_dict['chunk']
    chunk_size = es['games'].df.shape[0]
    feature_matrix = ft.calculate_feature_matrix(feature_defs, 
                                                     entityset=es, 
                                                     n_jobs = 1, 
                                                     verbose = 0,
                                                     chunk_size = chunk_size)   
    feature_matrix.to_parquet(data_folder+'p{}_fm_{}'.format(chunk, set_number), compression='SNAPPY')
    return True

es_dict = create_entityset({'chunk':0, 'path':data_folder+'p0'})
feature_defs_1 = ft.dfs(entityset=es_dict['es'], target_entity='games', 
                                          agg_primitives=['mean', 'count', 'std'], 
                                          trans_primitives=['absolute'], features_only=True, max_depth=2)
# time based computations take longer than non-temporal
feature_defs_2 = ft.dfs(entityset=es_dict['es'], target_entity='games', 
                                          agg_primitives=['mean', 'count', 'std'], 
                                          trans_primitives=['time_since', 'time_since_previous','diff'], features_only=True, max_depth=2)
feature_defs_3 = ft.dfs(entityset=es_dict['es'], target_entity='games', 
                                          agg_primitives=['median','avg_time_between','time_since_last'],
                                          trans_primitives=['absolute'], features_only=True, max_depth=2)
feature_defs_4 = ft.dfs(entityset=es_dict['es'], target_entity='games', 
                                          agg_primitives=['median','avg_time_between','time_since_last'], 
                                          trans_primitives=['time_since', 'time_since_previous','diff'], features_only=True, max_depth=2)
# skew and trend computations take significantly longer to compute
feature_defs_5 = ft.dfs(entityset=es_dict['es'], target_entity='games', 
                                          agg_primitives=['skew','trend'],
                                          trans_primitives=['absolute'], features_only=True, max_depth=2)
feature_defs_6 = ft.dfs(entityset=es_dict['es'], target_entity='games', 
                                          agg_primitives=['skew','trend'], 
                                          trans_primitives=['time_since', 'time_since_previous','diff'], features_only=True, max_depth=2)
print(len(feature_defs_1))
print(len(feature_defs_2))
print(len(feature_defs_3))
print(len(feature_defs_4))
print(len(feature_defs_5))
print(len(feature_defs_6))

315
415
215
306
301
399


In [12]:
from dask.distributed import Client, LocalCluster
cluster = LocalCluster(n_workers=4)
client = Client(cluster)
import dask.bag as db
paths = [{'chunk':i, 'path':data_folder+'p{}'.format(i)} for i in tqdm(range(total_chunks))]

b = db.from_sequence(paths)

b = b.map(create_entityset)

b = b.map(compute_features, feature_defs=feature_defs_6, set_number=6)
b

100%|██████████| 300/300 [00:00<00:00, 824028.29it/s]


dask.bag<compute..., npartitions=100>

In [ ]:
from dask.diagnostics import ProgressBar
ProgressBar().register()
b.compute()